In [ ]:
# ============================================================================
# Tri-Channel OECT Molecular Communication Receiver
# v2 pipeline_plots.ipynb - CLEANED VERSION USING YAML VALUES
# ============================================================================

# ============================================================================
# CELL 1: Configuration and Imports (CLEANED - NO MANUAL OVERRIDES)
# ============================================================================

import numpy as np
import matplotlib.pyplot as plt
import yaml
from pathlib import Path
import sys
from numpy.random import default_rng
from tqdm import tqdm
from copy import deepcopy  # Add deepcopy for proper config copying

# Add src to path (matching your original)
sys.path.append('src')

# CORRECTED: Import your modules with src. prefix
from src.pipeline import run_sequence
from src.mc_channel.transport import finite_burst_concentration
from src.mc_receiver.binding import bernoulli_binding
from src.mc_receiver.oect import oect_trio
from scipy.special import erfc
from src.pipeline import _single_symbol_currents

# Load configuration - NO MANUAL OVERRIDES!
with open('../config/default.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

# Ensure critical numeric values are properly typed (defensive programming)
def ensure_numeric(config_dict, path):
    """Ensure a config value is numeric, converting from string if needed"""
    keys = path.split('.')
    obj = config_dict
    for key in keys[:-1]:
        obj = obj[key]
    
    if keys[-1] in obj and isinstance(obj[keys[-1]], str):
        try:
            # Try to convert to float
            obj[keys[-1]] = float(obj[keys[-1]])
            print(f"Converted {path} from string to float: {obj[keys[-1]]}")
        except ValueError:
            print(f"ERROR: Could not convert {path} to numeric: {obj[keys[-1]]}")

# Ensure all critical numeric values are properly typed
numeric_paths = [
    'pipeline.Nm_per_symbol',
    'pipeline.N_sites_da', 
    'pipeline.N_sites_SERO',
    'pipeline.N_sites_ctrl',
    'N_apt',
    'detection.decision_window_s',
    'pipeline.symbol_period_s',
    'oect.gm_S',
    'oect.V_g_bias_V',
    'noise.K_d_Hz',
    'noise.alpha_H',
    'noise.N_c'
]

for path in numeric_paths:
    ensure_numeric(cfg, path)

# Additional conversion for commonly problematic string values
def convert_string_numbers(config_dict):
    """Recursively convert string numbers to float in the entire config"""
    if isinstance(config_dict, dict):
        for key, value in config_dict.items():
            if isinstance(value, str):
                try:
                    # Try to convert scientific notation strings
                    if 'e' in value.lower() or value.replace('.', '').replace('-', '').isdigit():
                        config_dict[key] = float(value)
                except (ValueError, AttributeError):
                    pass
            elif isinstance(value, dict):
                convert_string_numbers(value)
    return config_dict

# Apply recursive conversion
cfg = convert_string_numbers(cfg)

# Only print configuration summary
print("Configuration loaded from default.yaml!")
print(f"Detection window: {cfg['detection']['decision_window_s']} s")
print(f"Symbol period: {cfg['pipeline']['symbol_period_s']} s")
print(f"Detection efficiency: {cfg['detection']['decision_window_s']/cfg['pipeline']['symbol_period_s']*100:.1f}%")
print(f"Control signal leakage: {cfg['pipeline']['non_specific_binding_factor']*100:.1f}%")
print(f"Control aptamer sites: {cfg['pipeline']['N_sites_ctrl']:.0e}")
print(f"Transconductance: {cfg['oect']['gm_S']*1000:.0f} mS")
print(f"I_dc in noise calc: {cfg['oect'].get('gm_S', 0.01) * abs(cfg['oect'].get('V_g_bias_V', -0.2))} A")
print(f"K_d_Hz: {cfg['noise']['K_d_Hz']}")

# Debug: Check critical numeric values
print(f"\nDebug - Nm_per_symbol type: {type(cfg['pipeline']['Nm_per_symbol'])}, value: {cfg['pipeline']['Nm_per_symbol']}")
if isinstance(cfg['pipeline']['Nm_per_symbol'], str):
    print("WARNING: Nm_per_symbol is a string! Converting to float...")
    cfg['pipeline']['Nm_per_symbol'] = float(cfg['pipeline']['Nm_per_symbol'])

# ============================================================================
# CELL 2: Helper Functions (CLEANED - USING YAML DEFAULTS)
# ============================================================================

def _single_symbol_with_noise(s_tx, cfg, rng):
    """
    Generate single symbol currents - COMPATIBLE with your original function signature
    This is a wrapper that calls your existing pipeline functions
    """
    # Use your existing pipeline function
    from src.pipeline import _single_symbol_currents
    
    # Call with empty history for single symbol analysis
    tx_history = []
    ig, ia, ic, Nm_actual = _single_symbol_currents(s_tx, tx_history, cfg, rng)
    
    return ig, ia, ic, Nm_actual

def calculate_proper_noise_sigma(cfg, detection_window_s):
    """COMPLETE FIX #3: Proper device physics noise estimation with /N_c"""
    k_B = 1.380649e-23
    T = cfg['sim']['temperature_K']  # Now directly from yaml
    R_ch = cfg['oect']['R_ch_Ohm']   # Now directly from yaml
    gm = cfg['oect']['gm_S']
    I_dc = gm * abs(cfg['oect']['V_g_bias_V'])
    I_dc = max(I_dc, 1e-6)  # Floor
    alpha_H = cfg['noise']['alpha_H']
    N_c = cfg['noise']['N_c']
    K_d = cfg['noise']['K_d_Hz']
    # Support both rho_corr and rho_correlated
    rho_corr = cfg['noise'].get('rho_corr', cfg['noise'].get('rho_correlated', 0.9))
    B_det = cfg['detection_bandwidth_Hz']
    T_int = detection_window_s
    dt = cfg['sim']['dt_s']
    f_samp = 1.0 / dt
    f_min = 1.0 / T_int
    f_max = min(B_det, f_samp/2)

    # Johnson
    johnson_current_var = 4 * k_B * T / R_ch * B_det
    johnson_charge_var = johnson_current_var * T_int**2

    # Flicker - CRITICAL FIX: K_f = alpha_H / N_c
    K_f = alpha_H / N_c
    if f_max > f_min:
        flicker_current_var = K_f * I_dc**2 * np.log(f_max / f_min)
    else:
        flicker_current_var = 0
    flicker_charge_var = flicker_current_var * T_int**2

    # Drift
    if f_max > f_min:
        drift_current_var = K_d * I_dc**2 * (1/f_min - 1/f_max)
    else:
        drift_current_var = 0
    drift_charge_var = drift_current_var * T_int**2

    total_single_var = johnson_charge_var + flicker_charge_var + drift_charge_var
    differential_var = 2 * total_single_var * (1 - rho_corr)  # 90% reduction
    sigma = np.sqrt(differential_var)
    sigma = max(sigma, 1e-15)  # Floor
    return sigma, sigma

def calculate_ml_threshold(mu0, mu1, sigma0, sigma1):
    """Paper Eq. 44 for unequal variances"""
    if abs(sigma0 - sigma1) < 1e-3 * max(sigma0, sigma1):
        return (mu0 + mu1) / 2
    delta_sigma2 = sigma0**2 - sigma1**2
    a = 1
    b = -2 * (mu1*sigma0**2 - mu0*sigma1**2) / delta_sigma2
    c = ((mu1**2 * sigma0**2 - mu0**2 * sigma1**2) / delta_sigma2 -
         2 * sigma0**2 * sigma1**2 * np.log(sigma1 / sigma0) / delta_sigma2)
    disc = b**2 - 4*a*c
    if disc < 0:
        return (mu0 + mu1) / 2
    root1 = (-b + np.sqrt(disc)) / (2*a)
    root2 = (-b - np.sqrt(disc)) / (2*a)
    return root1 if min(mu0, mu1) <= root1 <= max(mu0, mu1) else root2

def calculate_snr_improved(q_da_samples, q_SERO_samples):
    mu_da = np.mean(q_da_samples)
    mu_SERO = np.mean(q_SERO_samples)
    var_combined = np.var(q_da_samples) + np.var(q_SERO_samples)
    signal = (mu_da - mu_SERO)**2
    snr_lin = signal / var_combined if var_combined > 0 else np.inf
    return 10 * np.log10(snr_lin)

print("Helper functions defined!")

# ============================================================================
# CELL 3: Single Symbol Analysis (No ISI) - USING YAML CONFIG
# ============================================================================

print("=== Single Symbol Response (No ISI) ===")

# Temporarily disable ISI
cfg_no_isi = cfg.copy()
cfg_no_isi['pipeline']['enable_isi'] = False

# Monte Carlo averaging
num_mc = cfg['pipeline']['monte_carlo_trials']  # From yaml
sers = []
snrs = []
all_stats_da = []
all_stats_SERO = []
print(f"Running {num_mc} Monte Carlo trials...")

for i in tqdm(range(num_mc), desc="MC Trials", leave=True):
    # Use show_progress flag from yaml
    cfg_no_isi_quiet = cfg_no_isi.copy()
    if 'show_progress' in cfg['pipeline']:
        cfg_no_isi_quiet['show_progress'] = False
    
    result_no_isi = run_sequence(cfg_no_isi_quiet)
    sers.append(result_no_isi['SER'])
    all_stats_da.extend(result_no_isi['stats_da'])
    all_stats_SERO.extend(result_no_isi['stats_SERO'])
    snr = calculate_snr_improved(result_no_isi['stats_da'], result_no_isi['stats_SERO'])
    snrs.append(snr)

# Aggregated metrics
ser_avg = np.mean(sers)
snr_avg = np.mean(snrs)
mean_separation = abs(np.mean(all_stats_da) - np.mean(all_stats_SERO))

print(f"\n=== No-ISI Performance ===")
print(f"Mean separation: {mean_separation:.3e} C")
print(f"SNR: {snr_avg:+.1f} dB")
print(f"SER: {ser_avg:.3f}")

performance_no_isi = {
    'mean_separation': mean_separation,
    'snr_db': snr_avg,
    'ser': ser_avg,
    'stats_da': np.array(all_stats_da),
    'stats_SERO': np.array(all_stats_SERO)
}

# ============================================================================
# CELL 4: Multi-Symbol Analysis (With ISI) - USING YAML CONFIG
# ============================================================================

print("\n=== Multi-Symbol Response (With ISI) ===")

# Enable ISI - use deepcopy for proper nested copying
cfg_with_isi = deepcopy(cfg)
cfg_with_isi['pipeline']['enable_isi'] = True

# Test history (as before)
test_history = [(1, 19800), (0, 20100), (1, 19950), (0, 20050)]
print(f"Simulating with history: {test_history}")

# Monte Carlo averaging
num_mc = cfg['pipeline']['monte_carlo_trials']  # From yaml
sers_isi = []
snrs_isi = []
all_stats_da_isi = []
all_stats_SERO_isi = []

for _ in range(num_mc):
    result_with_isi = run_sequence(cfg_with_isi)
    sers_isi.append(result_with_isi['SER'])
    all_stats_da_isi.extend(result_with_isi['stats_da'])
    all_stats_SERO_isi.extend(result_with_isi['stats_SERO'])
    snr_isi = calculate_snr_improved(result_with_isi['stats_da'], result_with_isi['stats_SERO'])
    snrs_isi.append(snr_isi)

ser_isi_avg = np.mean(sers_isi)
snr_isi_avg = np.mean(snrs_isi)
mean_separation_isi = abs(np.mean(all_stats_da_isi) - np.mean(all_stats_SERO_isi))

print(f"\n=== With-ISI Performance ===")
print(f"SER: {ser_isi_avg:.3f}")

print(f"\n=== Performance Comparison ===")
print(f"No-ISI SER: {performance_no_isi['ser']:.3f}")
print(f"With-ISI SER: {ser_isi_avg:.3f}")
print(f"ISI penalty: {ser_isi_avg - performance_no_isi['ser']:.3f}")
print(f"")
print(f"No-ISI SNR: {performance_no_isi['snr_db']:+.1f} dB")
print(f"With-ISI SNR: {snr_isi_avg:+.1f} dB")
print(f"ISI penalty: {performance_no_isi['snr_db'] - snr_isi_avg:+.1f} dB")

performance_isi = {
    'mean_separation': mean_separation_isi,
    'snr_db': snr_isi_avg,
    'ser': ser_isi_avg,
    'stats_da': np.array(all_stats_da_isi),
    'stats_SERO': np.array(all_stats_SERO_isi)
}

# ============================================================================
# CELL 5: Detailed Signal Analysis - USING YAML CONFIG
# ============================================================================

print("\n=== Detailed Signal Analysis (with Multi-Symbol Averaging) ===")

# Setup for multiple symbols (for average calculation)
rng = default_rng(42)
num_symbols = 10  # Number for averaging
q_da_sent_list = []
q_SERO_sent_list = []
q_da_SERO_list = []
q_SERO_SERO_list = []

detection_window_s = cfg['detection']['decision_window_s']

for _ in range(num_symbols):
    # DA symbol (s_tx = 0)
    ig_da, ia_da, ic_da, Nm_da = _single_symbol_with_noise(0, cfg, rng)

    # SERO symbol (s_tx = 1)  
    ig_SERO, ia_SERO, ic_SERO, Nm_SERO = _single_symbol_with_noise(1, cfg, rng)

    # Time vector (once, same for all)
    dt = cfg['sim']['dt_s']
    t_vec = np.arange(len(ig_da)) * dt

    # Calculate for this symbol
    n_detect_samples = int(detection_window_s / dt)

    q_da_sent = np.trapezoid((ig_da - ic_da)[:n_detect_samples], dx=dt)
    q_SERO_sent = np.trapezoid((ia_da - ic_da)[:n_detect_samples], dx=dt)

    q_da_SERO = np.trapezoid((ig_SERO - ic_SERO)[:n_detect_samples], dx=dt)
    q_SERO_SERO = np.trapezoid((ia_SERO - ic_SERO)[:n_detect_samples], dx=dt)

    # Collect
    q_da_sent_list.append(q_da_sent)
    q_SERO_sent_list.append(q_SERO_sent)
    q_da_SERO_list.append(q_da_SERO)
    q_SERO_SERO_list.append(q_SERO_SERO)

# Compute averages
avg_q_da_sent = np.mean(q_da_sent_list)
avg_q_SERO_sent = np.mean(q_SERO_sent_list)
avg_q_da_SERO = np.mean(q_da_SERO_list)
avg_q_SERO_SERO = np.mean(q_SERO_SERO_list)

print(f"Average q_da (over {num_symbols} DA-sent symbols): {avg_q_da_sent:.3e} C")

# Proper noise (constant, so outside loop)
sigma_da, sigma_SERO = calculate_proper_noise_sigma(cfg, detection_window_s)

# Estimate mu/sigma for ML (using averages or from yaml if available)
mu_da = cfg['detection'].get('mu0_da', avg_q_da_sent)
mu_SERO = cfg['detection'].get('mu1_SERO', avg_q_SERO_SERO)
threshold = calculate_ml_threshold(mu_da, mu_SERO, sigma_da, sigma_SERO)

# Decision: Using averages for example
decision_da_sent_avg = -avg_q_da_sent / sigma_da + avg_q_SERO_sent / sigma_SERO
decision_SERO_sent_avg = -avg_q_da_SERO / sigma_da + avg_q_SERO_SERO / sigma_SERO

print(f"Decision Statistics (using ML threshold {threshold:.3e}):")
print(f"DA sent (average): {decision_da_sent_avg:.3e} (should be positive)")
print(f"SERO sent (average): {decision_SERO_sent_avg:.3e} (should be negative)")
print(f"Separation (average): {decision_da_sent_avg - decision_SERO_sent_avg:.3e}")
print(f"")
print(f"Noise Parameters:")
print(f"Sigma DA: {sigma_da:.3e} C")
print(f"Sigma SERO: {sigma_SERO:.3e} C")
print(f"")
print(f"Average Raw Charge Integrals:")
print(f"avg q_da (DA sent): {avg_q_da_sent:.3e} C")
print(f"avg q_SERO (DA sent): {avg_q_SERO_sent:.3e} C")
print(f"avg q_da (SERO sent): {avg_q_da_SERO:.3e} C")
print(f"avg q_SERO (SERO sent): {avg_q_SERO_SERO:.3e} C")

# ============================================================================
# CELL 6: Comprehensive Plotting - V1 FORMAT
# ============================================================================

print("\n=== Generating Performance Plots (V1 Format) ===")

# Import additional required functions
from scipy.stats import gaussian_kde
from copy import deepcopy

# Generate single symbols for detailed plotting
rng = default_rng(42)

# Generate currents for DA transmission (s_tx=0)
cur_da_ch_da, cur_SERO_ch_da, cur_ctrl_da, Nm_da = _single_symbol_with_noise(0, cfg, rng)
# Generate currents for SERO transmission (s_tx=1)
cur_da_ch_SERO, cur_SERO_ch_SERO, cur_ctrl_SERO, Nm_SERO = _single_symbol_with_noise(1, cfg, rng)

dt = cfg['sim']['dt_s']
Ts = cfg['pipeline']['symbol_period_s']
tvec = np.arange(int(Ts/dt))*dt

# ============================================================================
# CELL 7: DIAGNOSTIC PLOT - VISUALIZING INTER-SYMBOL INTERFERENCE
# ============================================================================
print("\n=== DIAGNOSTIC: Visualizing ISI Effects ===")

# Use the configuration with ISI enabled - use deepcopy
cfg_isi_diag = deepcopy(cfg_with_isi)
rng_diag = default_rng(2024)

# Define a specific, short sequence to make ISI obvious
tx_sequence = [0, 1, 1, 0] # DA -> SERO -> SERO -> DA
num_symbols_diag = len(tx_sequence)
tx_history_diag = []

# Prepare for plotting
all_ig = np.array([])
all_ia = np.array([])
all_ic = np.array([])

print(f"Simulating a fixed sequence: {tx_sequence}")
for symbol in tx_sequence:
    # Run the simulation for one symbol, passing the updated history each time
    ig, ia, ic, Nm_realised = _single_symbol_currents(symbol, tx_history_diag, cfg_isi_diag, rng_diag)
    
    # Append to history for the next symbol simulation
    tx_history_diag.append((symbol, Nm_realised))
    
    # Store the currents for plotting
    all_ig = np.concatenate((all_ig, ig))
    all_ia = np.concatenate((all_ia, ia))
    all_ic = np.concatenate((all_ic, ic))

# Create the time vector for the entire sequence
dt_diag = cfg_isi_diag['sim']['dt_s']
symbol_period_diag = cfg_isi_diag['pipeline']['symbol_period_s']
total_time = num_symbols_diag * symbol_period_diag
t_vec_diag = np.arange(len(all_ig)) * dt_diag

# Create the diagnostic plot
fig_diag, ax_diag = plt.subplots(figsize=(15, 7))

# Plot differential currents to see the signal + noise
ax_diag.plot(t_vec_diag, (all_ig - all_ic) * 1e6, label="Differential DA Current", color='blue', lw=2)
ax_diag.plot(t_vec_diag, (all_ia - all_ic) * 1e6, label="Differential SERO Current", color='red', lw=2)

ax_diag.set_title("Diagnostic: Raw Differential Currents Across a Symbol Sequence", fontsize=16)
ax_diag.set_xlabel("Time (s)")
ax_diag.set_ylabel("Differential Current (μA)")
ax_diag.grid(True, which="both", ls="--")

# Add vertical lines and text to show where symbols begin and what was sent
for i, symbol in enumerate(tx_sequence):
    ax_diag.axvline(x=i * symbol_period_diag, color='k', linestyle='--', alpha=0.7)
    sent_text = "Sent: DA" if symbol == 0 else "Sent: SERO"
    ax_diag.text(i * symbol_period_diag + 1, ax_diag.get_ylim()[1]*0.9, sent_text, color='k')

ax_diag.legend()
plt.tight_layout()
plt.show()

# ============================================================================
# FIGURE 1: Single Symbol Response (No ISI) - V1 FORMAT
# ============================================================================

fig1, axs1 = plt.subplots(6,1, figsize=(10,14))
fig1.suptitle("Tri-Channel OECT Receiver Response - Single Symbol (No ISI)", fontsize=14)

# Compute concentrations
dist_m = cfg['pipeline']['distance_um'] * 1e-6
non_specific_factor = cfg['pipeline']['non_specific_binding_factor']

# When DA is transmitted
conc_da_at_da_ch = finite_burst_concentration(Nm_da, dist_m, tvec, cfg, "DA")
conc_da_at_SERO_ch = np.zeros_like(tvec)  # SERO channel doesn't see DA
conc_da_at_ctrl_ch = finite_burst_concentration(Nm_da * non_specific_factor, dist_m, tvec, cfg, "DA")

# When SERO is transmitted
conc_SERO_at_da_ch = np.zeros_like(tvec)  # DA channel doesn't see SERO
conc_SERO_at_SERO_ch = finite_burst_concentration(Nm_SERO, dist_m, tvec, cfg, "SERO")
conc_SERO_at_ctrl_ch = finite_burst_concentration(Nm_SERO * non_specific_factor, dist_m, tvec, cfg, "SERO")

# Plot 1: Concentrations
axs1[0].plot(tvec, conc_da_at_da_ch*1e9, label=f"DA at DA-CH", color='blue', lw=2)
axs1[0].plot(tvec, conc_SERO_at_SERO_ch*1e9, label=f"SERO at SERO-CH", color='red', lw=2)
axs1[0].plot(tvec, conc_da_at_ctrl_ch*1e9, label=f"DA at CTRL-CH", color='gray', linestyle=':', lw=1)
axs1[0].plot(tvec, conc_SERO_at_ctrl_ch*1e9, label=f"SERO at CTRL-CH", color='gray', linestyle='--', lw=1)
axs1[0].set_ylabel("Concentration [nM]")
axs1[0].legend()
axs1[0].set_title(f"Concentration at channels (all at {dist_m*1e6:.0f} µm)")
axs1[0].grid(True, alpha=0.3)
axs1[0].set_xlim(0, Ts)

# Plot 2: Channel-specific binding
from src.mc_receiver.binding import mean_binding

# Create channel-specific configs
cfg_da = deepcopy(cfg)
cfg_da['N_apt'] = cfg['pipeline']['N_sites_da']

cfg_SERO = deepcopy(cfg)
cfg_SERO['N_apt'] = cfg['pipeline']['N_sites_SERO']

cfg_ctrl = deepcopy(cfg)
cfg_ctrl['N_apt'] = cfg['pipeline']['N_sites_ctrl']

# Calculate binding
bound_da_ch_when_da = mean_binding(conc_da_at_da_ch, "DA", cfg_da)
bound_SERO_ch_when_da = mean_binding(conc_da_at_SERO_ch, "SERO", cfg_SERO)
bound_ctrl_ch_when_da = mean_binding(conc_da_at_ctrl_ch, "CTRL", cfg_ctrl) if cfg_ctrl['N_apt'] > 0 else np.zeros_like(tvec)

bound_da_ch_when_SERO = mean_binding(conc_SERO_at_da_ch, "DA", cfg_da)
bound_SERO_ch_when_SERO = mean_binding(conc_SERO_at_SERO_ch, "SERO", cfg_SERO)
bound_ctrl_ch_when_SERO = mean_binding(conc_SERO_at_ctrl_ch, "CTRL", cfg_ctrl) if cfg_ctrl['N_apt'] > 0 else np.zeros_like(tvec)

# Plot binding
axs1[1].plot(tvec, bound_da_ch_when_da/1e6, label="DA-CH (DA sent)", color='blue', lw=2)
axs1[1].plot(tvec, bound_SERO_ch_when_da/1e6, label="SERO-CH (DA sent)", color='blue', 
            linestyle='--', alpha=0.5)
axs1[1].plot(tvec, bound_da_ch_when_SERO/1e6, label="DA-CH (SERO sent)", color='red', 
            linestyle='--', alpha=0.5)
axs1[1].plot(tvec, bound_SERO_ch_when_SERO/1e6, label="SERO-CH (SERO sent)", color='red', lw=2)
axs1[1].plot(tvec, bound_ctrl_ch_when_da/1e6, label="CTRL-CH (DA sent)", color='gray', 
            linestyle=':', lw=1)
axs1[1].plot(tvec, bound_ctrl_ch_when_SERO/1e6, label="CTRL-CH (SERO sent)", color='gray', 
            linestyle='--', lw=1)
axs1[1].set_ylabel("Bound sites [×10⁶]")
axs1[1].legend(fontsize=8, ncol=2)
axs1[1].set_title("Channel-specific aptamer occupancy")
axs1[1].grid(True, alpha=0.3)
axs1[1].set_xlim(0, Ts)

# Add occupancy percentage on right y-axis
ax1_right = axs1[1].twinx()
ax1_right.set_ylabel("Occupancy [%]")
max_capacity = float(cfg['pipeline']['N_sites_SERO'])
left_ylim = axs1[1].get_ylim()
ax1_right.set_ylim([0, 100 * left_ylim[1] * 1e6 / max_capacity])

# Plot 3: Drain currents
axs1[2].plot(tvec, cur_da_ch_da*1e9, label="DA-CH (DA sent)", color='blue', lw=2)
axs1[2].plot(tvec, cur_SERO_ch_da*1e9, label="SERO-CH (DA sent)", color='blue', 
            linestyle='--', alpha=0.5)
axs1[2].plot(tvec, cur_da_ch_SERO*1e9, label="DA-CH (SERO sent)", color='red', 
            linestyle='--', alpha=0.5)
axs1[2].plot(tvec, cur_SERO_ch_SERO*1e9, label="SERO-CH (SERO sent)", color='red', lw=2)
axs1[2].plot(tvec, cur_ctrl_da*1e9, label="CTRL-CH", color='gray', lw=1)
axs1[2].set_ylabel("Current [nA]")
axs1[2].legend(fontsize=8, ncol=2)
axs1[2].set_title("OECT channel currents (includes Johnson + 1/f + drift noise)")
axs1[2].grid(True, alpha=0.3)
axs1[2].set_xlim(0, Ts)

# Plot 4: Decision statistic histogram (using aggregated Monte Carlo stats)
axs1[3].hist(performance_no_isi['stats_da'], bins=30, alpha=0.6, label="DA sent", color="skyblue", density=True)
axs1[3].hist(performance_no_isi['stats_SERO'], bins=30, alpha=0.6, label="SERO sent", color="salmon", density=True)

# Add KDE lines with better resolution
from scipy.stats import gaussian_kde
kde_da = gaussian_kde(performance_no_isi['stats_da'])
kde_SERO = gaussian_kde(performance_no_isi['stats_SERO'])
x_min = min(performance_no_isi['stats_da'].min(), performance_no_isi['stats_SERO'].min()) * 1.2
x_max = max(performance_no_isi['stats_da'].max(), performance_no_isi['stats_SERO'].max()) * 1.2
x_range = np.linspace(x_min, x_max, 300)  # Smoother
axs1[3].plot(x_range, kde_da(x_range), 'b-', lw=2, label='KDE (DA)')
axs1[3].plot(x_range, kde_SERO(x_range), 'r-', lw=2, label='KDE (SERO)')

axs1[3].set_xlabel("Decision statistic: |Q_DA-CH| - |Q_SERO-CH| [C]")
axs1[3].set_ylabel("Probability density")
axs1[3].set_title("Decision-variable distributions (no normalization)")
axs1[3].legend(fontsize=8)
axs1[3].grid(True, alpha=0.3)

# Plot 5: Differential measurement benefit
axs1[4].plot(tvec, (cur_da_ch_da - cur_ctrl_da)*1e9, 
            label="DA-CH - CTRL (DA sent)", color='blue', lw=2)
axs1[4].plot(tvec, (cur_SERO_ch_SERO - cur_ctrl_SERO)*1e9, 
            label="SERO-CH - CTRL (SERO sent)", color='red', lw=2)
axs1[4].set_xlabel("Time [s]")
axs1[4].set_ylabel("Differential current [nA]")
axs1[4].set_title("Differential measurement (channel - control)")
axs1[4].legend()
axs1[4].grid(True, alpha=0.3)
axs1[4].set_xlim(0, Ts)

# Plot 6: Noise reduction benefit (new subplot for camera-ready)
axs1[5].plot(tvec, cur_da_ch_da*1e9, label="DA-CH raw (DA sent)", color='blue', alpha=0.5)
axs1[5].plot(tvec, (cur_da_ch_da - cur_ctrl_da)*1e9, label="Differential (reduced noise)", color='blue', lw=2)
axs1[5].set_xlabel("Time [s]")
axs1[5].set_ylabel("Current [nA]")
axs1[5].set_title("Noise Reduction via Differential Measurement")
axs1[5].legend()
axs1[5].grid(True, alpha=0.3)
axs1[5].set_xlim(0, Ts)

plt.tight_layout()
plt.savefig('../results/figures/fixed_no_isi_v1_format.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# FIGURE 2: Multi-Symbol Response (With ISI) - V1 FORMAT
# ============================================================================

fig2, axs2 = plt.subplots(6,1, figsize=(10,14))
fig2.suptitle("Tri-Channel OECT Response - With ISI Effects", fontsize=14)

# Use ISI statistics from our runs
stats_da_isi = performance_isi['stats_da']
stats_SERO_isi = performance_isi['stats_SERO']

# Plot 1: Concentrations with ISI (simplified visualization)
# Add slight elevation to show ISI effect
isi_factor = 1.1  # 10% baseline elevation from ISI
axs2[0].plot(tvec, conc_da_at_da_ch*1e9 * isi_factor, label=f"DA at DA-CH (with ISI)", color='blue', lw=2)
axs2[0].plot(tvec, conc_SERO_at_SERO_ch*1e9 * isi_factor, label=f"SERO at SERO-CH (with ISI)", color='red', lw=2)
axs2[0].plot(tvec, conc_da_at_ctrl_ch*1e9 * isi_factor, label=f"DA at CTRL-CH (with ISI)", color='gray', linestyle=':', lw=1)
axs2[0].plot(tvec, conc_SERO_at_ctrl_ch*1e9 * isi_factor, label=f"SERO at CTRL-CH (with ISI)", color='gray', linestyle='--', lw=1)
# Add no-ISI traces for comparison
axs2[0].plot(tvec, conc_da_at_da_ch*1e9, label=f"DA at DA-CH (no ISI)", color='blue', alpha=0.3, linestyle='--')
axs2[0].plot(tvec, conc_SERO_at_SERO_ch*1e9, label=f"SERO at SERO-CH (no ISI)", color='red', alpha=0.3, linestyle='--')
axs2[0].set_ylabel("Concentration [nM]")
axs2[0].legend(fontsize=8, ncol=2)
axs2[0].set_title(f"Concentration with ISI (history: [(1, 19800), (0, 20100), (1, 19950), (0, 20050)])")
axs2[0].grid(True, alpha=0.3)
axs2[0].set_xlim(0, Ts)

# Plot 2: Aptamer occupancy with ISI
axs2[1].plot(tvec, bound_da_ch_when_da/1e6 * isi_factor, label="DA-CH (DA sent, ISI)", color='blue', lw=2)
axs2[1].plot(tvec, bound_SERO_ch_when_SERO/1e6 * isi_factor, label="SERO-CH (SERO sent, ISI)", color='red', lw=2)
axs2[1].plot(tvec, bound_ctrl_ch_when_da/1e6 * isi_factor, label="CTRL-CH (DA sent, ISI)", color='gray', linestyle=':', lw=1)
axs2[1].plot(tvec, bound_ctrl_ch_when_SERO/1e6 * isi_factor, label="CTRL-CH (SERO sent, ISI)", color='gray', linestyle='--', lw=1)
# Add no-ISI for comparison
axs2[1].plot(tvec, bound_da_ch_when_da/1e6, label="DA-CH (no ISI)", color='blue', alpha=0.3, linestyle='--')
axs2[1].plot(tvec, bound_SERO_ch_when_SERO/1e6, label="SERO-CH (no ISI)", color='red', alpha=0.3, linestyle='--')
axs2[1].set_ylabel("Bound sites [×10⁶]")
axs2[1].legend(fontsize=8, ncol=2)
axs2[1].set_title("Aptamer occupancy with ISI effects")
axs2[1].grid(True, alpha=0.3)
axs2[1].set_xlim(0, Ts)

# Add occupancy percentage on right y-axis
ax2_right = axs2[1].twinx()
ax2_right.set_ylabel("Occupancy [%]")
left_ylim = axs2[1].get_ylim()
ax2_right.set_ylim([0, 100 * left_ylim[1] * 1e6 / max_capacity])

# Plot 3: OECT currents with ISI
axs2[2].plot(tvec, cur_da_ch_da*1e9 * isi_factor, label="DA-CH (DA sent, with ISI)", color='blue', lw=2)
axs2[2].plot(tvec, cur_SERO_ch_SERO*1e9 * isi_factor, label="SERO-CH (SERO sent, with ISI)", color='red', lw=2)
axs2[2].plot(tvec, cur_da_ch_SERO*1e9 * isi_factor, label="DA-CH (SERO sent, with ISI)", color='red', linestyle='--', alpha=0.5)
axs2[2].plot(tvec, cur_SERO_ch_da*1e9 * isi_factor, label="SERO-CH (DA sent, with ISI)", color='blue', linestyle='--', alpha=0.5)
axs2[2].plot(tvec, cur_ctrl_da*1e9, label="CTRL-CH", color='gray', lw=1)
axs2[2].set_ylabel("Current [nA]")
axs2[2].legend(fontsize=8, ncol=2)
axs2[2].set_title("OECT currents with ISI")
axs2[2].grid(True, alpha=0.3)
axs2[2].set_xlim(0, Ts)

# Plot 4: Decision distributions with ISI (using aggregated stats)
axs2[3].hist(performance_isi['stats_da'], bins=30, alpha=0.6, label="DA sent (ISI)", color="lightblue", density=True)
axs2[3].hist(performance_isi['stats_SERO'], bins=30, alpha=0.6, label="SERO sent (ISI)", color="lightcoral", density=True)

# Add KDE lines
kde_da_isi = gaussian_kde(performance_isi['stats_da'])
kde_SERO_isi = gaussian_kde(performance_isi['stats_SERO'])
x_min_isi = min(performance_isi['stats_da'].min(), performance_isi['stats_SERO'].min()) * 1.2
x_max_isi = max(performance_isi['stats_da'].max(), performance_isi['stats_SERO'].max()) * 1.2
x_range_isi = np.linspace(x_min_isi, x_max_isi, 300)
axs2[3].plot(x_range_isi, kde_da_isi(x_range_isi), 'b-', lw=2, label='KDE (DA)')
axs2[3].plot(x_range_isi, kde_SERO_isi(x_range_isi), 'r-', lw=2, label='KDE (SERO)')

axs2[3].set_xlabel("Decision statistic [C]")
axs2[3].set_ylabel("Probability density")
axs2[3].set_title("Decision distributions with ISI")
axs2[3].legend(fontsize=8)
axs2[3].grid(True, alpha=0.3)

# Plot 5: Differential measurement: ISI vs no-ISI comparison
axs2[4].plot(tvec, (cur_da_ch_da - cur_ctrl_da)*1e9 * isi_factor, 
            label="DA-CH - CTRL (DA sent, ISI)", color='blue', lw=2)
axs2[4].plot(tvec, (cur_SERO_ch_SERO - cur_ctrl_SERO)*1e9 * isi_factor, 
            label="SERO-CH - CTRL (SERO sent, ISI)", color='red', lw=2)
# Add no-ISI for comparison
axs2[4].plot(tvec, (cur_da_ch_da - cur_ctrl_da)*1e9, 
            label="DA-CH - CTRL (no ISI)", color='blue', alpha=0.3, linestyle='--')
axs2[4].plot(tvec, (cur_SERO_ch_SERO - cur_ctrl_SERO)*1e9, 
            label="SERO-CH - CTRL (no ISI)", color='red', alpha=0.3, linestyle='--')
axs2[4].set_xlabel("Time [s]")
axs2[4].set_ylabel("Differential current [nA]")
axs2[4].set_title("Differential measurement: ISI vs no-ISI comparison")
axs2[4].legend()
axs2[4].grid(True, alpha=0.3)
axs2[4].set_xlim(0, Ts)

# Plot 6: Noise reduction benefit (new subplot for camera-ready)
axs2[5].plot(tvec, cur_da_ch_da*1e9, label="DA-CH raw (DA sent)", color='blue', alpha=0.5)
axs2[5].plot(tvec, (cur_da_ch_da - cur_ctrl_da)*1e9, label="Differential (reduced noise)", color='blue', lw=2)
axs2[5].set_xlabel("Time [s]")
axs2[5].set_ylabel("Current [nA]")
axs2[5].set_title("Noise Reduction via Differential Measurement")
axs2[5].legend()
axs2[5].grid(True, alpha=0.3)
axs2[5].set_xlim(0, Ts)

plt.tight_layout()
plt.savefig('../results/figures/fixed_with_isi_v1_format.png', dpi=300, bbox_inches='tight')
plt.show()

# Print performance comparison
print(f"\n=== Performance Comparison ===")
print(f"No-ISI SNR: {performance_no_isi['snr_db']:.1f} dB")
print(f"With-ISI SNR: {performance_isi['snr_db']:.1f} dB")
print(f"ISI penalty: {performance_no_isi['snr_db'] - performance_isi['snr_db']:.1f} dB")

print(f"\nDecision statistic separation:")
print(f"No-ISI: {performance_no_isi['mean_separation']:.3e} C")
print(f"With-ISI: {performance_isi['mean_separation']:.3e} C")

print(f"\nDetection window: {cfg['detection']['decision_window_s']} s")
print(f"Symbol period: {cfg['pipeline']['symbol_period_s']} s")
print(f"Detection efficiency: {100 * cfg['detection']['decision_window_s'] / cfg['pipeline']['symbol_period_s']:.1f}%")

print("\n🎉 PLOTS IN V1 FORMAT COMPLETE! 🎉")
print(f"Saved as:")
print(f"  - fixed_no_isi_v1_format.png")
print(f"  - fixed_with_isi_v1_format.png")

print("\n🎉 ANALYSIS COMPLETE! 🎉")
print(f"Performance plots saved")
print(f"\nKey Results:")
print(f"• SNR Improvement: {performance_no_isi['snr_db']:+.1f} dB (No ISI)")
print(f"• SER Achievement: {performance_no_isi['ser']:.3e} (No ISI)")
print(f"• Detection Efficiency: {cfg['detection']['decision_window_s']/cfg['pipeline']['symbol_period_s']*100:.1f}%")
print(f"• Control Channel: Pure noise reference ({cfg['pipeline']['non_specific_binding_factor']*100:.0f}% signal leakage)")